In [ ]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 7.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=dfa3784cc8b458932adf065c6d7c0907c672bad1c8522353f8241a60be77298f
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [ ]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from konlpy.tag import Mecab
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
from pandas import DataFrame 
import numpy as np
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_2018 = pd.read_csv('/content/drive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/PreprocessedData/2018_noun.csv')
data_2019 = pd.read_csv('/content/drive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/PreprocessedData/2019_noun.csv')
data_2020 = pd.read_csv('/content/drive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/PreprocessedData/2020_noun.csv')
data_2021 = pd.read_csv('/content/drive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/PreprocessedData/2021_noun.csv')

In [ ]:
data_2018 = data_2018[data_2018['press'].isin(['조선일보', '동아일보', '한겨레', '경향신문'])]
data_2019 = data_2019[data_2019['press'].isin(['조선일보', '동아일보', '한겨레', '경향신문'])]
data_2020 = data_2020[data_2020['press'].isin(['조선일보', '동아일보', '한겨레', '경향신문'])]
data_2021 = data_2021[data_2021['press'].isin(['조선일보', '동아일보', '한겨레', '경향신문'])]

data_2018.press.unique()

array(['한겨레', '경향신문', '조선일보', '동아일보'], dtype=object)

#2018

In [ ]:
first_half = data_2018['month'] <= 6
data_2018_fh = data_2018[first_half]

second_half = data_2018['month'] > 6
data_2018_sh = data_2018[second_half]

## 2018 상반기

In [ ]:
import ast
data_2018_fh_list = []
for content in data_2018_fh['noun_content']:
  content = ast.literal_eval(content)

import ast
data_2018_fh_list = []
for i in range(len(data_2018_fh)):
  tmp = data_2018_fh.iloc[i]
  c = ast.literal_eval(tmp.noun_content)
  content = [i for i in c if len(i)!=1]
  data_2018_fh_list.append(content)

dictionary = corpora.Dictionary(data_2018_fh_list)

# 빈도가 2 이상인 단어와 전체의 50%로 이상 차지하는 단어는 필터링
dictionary.filter_extremes(no_below=2, no_above=0.5)
#dictionary 속 단어가 문장에서 몇 번 출현하는지 빈도를 세서 벡터화(bag of words)이렇게 구조화된 언어를 코퍼스라고 합니다.
corpus = [dictionary.doc2bow(text) for text in data_2018_fh_list]

### 모델링

In [ ]:
num_topics = 20
chunksize = 2000
passes = 5
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

In [ ]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'LDA_2018_상반기.html')

## 2018 하반기

In [ ]:
import ast
data_2018_sh_list = []
for content in data_2018_sh['noun_content']:
  content = ast.literal_eval(content)

import ast
data_2018_sh_list = []
for i in range(len(data_2018_sh)):
  tmp = data_2018_sh.iloc[i]
  c = ast.literal_eval(tmp.noun_content)
  content = [i for i in c if len(i)!=1]
  data_2018_sh_list.append(content)

dictionary = corpora.Dictionary(data_2018_sh_list)

# 빈도가 2 이상인 단어와 전체의 50%로 이상 차지하는 단어는 필터링
dictionary.filter_extremes(no_below=2, no_above=0.5)
#dictionary 속 단어가 문장에서 몇 번 출현하는지 빈도를 세서 벡터화(bag of words)이렇게 구조화된 언어를 코퍼스라고 합니다.
corpus = [dictionary.doc2bow(text) for text in data_2018_sh_list]

2022-05-25 08:34:53,950 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-25 08:34:54,367 : INFO : adding document #10000 to Dictionary(21964 unique tokens: ['강사', '과거', '관심', '뉴델리', '다혜']...)
2022-05-25 08:34:54,780 : INFO : adding document #20000 to Dictionary(29484 unique tokens: ['강사', '과거', '관심', '뉴델리', '다혜']...)
2022-05-25 08:34:55,212 : INFO : adding document #30000 to Dictionary(34933 unique tokens: ['강사', '과거', '관심', '뉴델리', '다혜']...)
2022-05-25 08:34:55,623 : INFO : adding document #40000 to Dictionary(39308 unique tokens: ['강사', '과거', '관심', '뉴델리', '다혜']...)
2022-05-25 08:34:56,042 : INFO : adding document #50000 to Dictionary(43147 unique tokens: ['강사', '과거', '관심', '뉴델리', '다혜']...)
2022-05-25 08:34:56,493 : INFO : adding document #60000 to Dictionary(46332 unique tokens: ['강사', '과거', '관심', '뉴델리', '다혜']...)
2022-05-25 08:34:56,915 : INFO : adding document #70000 to Dictionary(49360 unique tokens: ['강사', '과거', '관심', '뉴델리', '다혜']...)
2022-05-25 08:34:57,361 

### 모델링

In [ ]:
num_topics = 20
chunksize = 2000
passes = 5
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2022-05-25 08:35:01,397 : INFO : using autotuned alpha, starting with [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
2022-05-25 08:35:01,409 : INFO : using serial LDA version on this node
2022-05-25 08:35:01,508 : INFO : running online (multi-pass) LDA training, 20 topics, 5 passes over the supplied corpus of 86465 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-05-25 08:35:01,510 : INFO : PROGRESS: pass 0, at document #2000/86465
2022-05-25 08:35:03,827 : INFO : optimized alpha [0.04206826, 0.0413232, 0.042904474, 0.042959005, 0.04331866, 0.040804297, 0.041981593, 0.042541683, 0.04233707, 0.042067893, 0.042277124, 0.04185753, 0.041912854, 0.042817056, 0.042276725, 0.04242964, 0.04257895, 0.042254053, 0.04280949, 0.042336963]
2022-05-25 08:35:03,833 : INFO : merging changes from 2000 documents into a model o

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-05-25 08:38:01,826 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-05-25 08:38:01,848 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-05-25 08:38:01,865 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-05-25 08:38:01,884 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-05-25 08:38:01,901 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-05-25 08:38:01,920 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-05-25 08:38:01,937 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2022-05-25 08:38:01,955 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2022-05-25 08:38:01,974 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2022-05-25 08:38:01,994 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2022-05-25 08:38:02,016 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2022-05-25 08:38:02

Average topic coherence: -4.9582.
[([(0.05195201, '북한'),
   (0.040848162, '미국'),
   (0.028950172, '트럼프'),
   (0.026854472, '남북'),
   (0.023243515, '대통령'),
   (0.017328044, '현지'),
   (0.015498386, '김정은'),
   (0.014037396, '도널드'),
   (0.012473311, '대북'),
   (0.011084408, '협상'),
   (0.010981698, '시각'),
   (0.010930495, '공동'),
   (0.01070255, '한반도'),
   (0.010540239, '정상회담'),
   (0.010521807, '평화'),
   (0.010396897, '비핵화'),
   (0.009759083, '국무위원'),
   (0.009571171, '제재'),
   (0.009338311, '합의'),
   (0.008764828, '시간')],
  -2.287799788358381),
 ([(0.050411798, '혐의'),
   (0.043457653, '검찰'),
   (0.027609004, '사건'),
   (0.02664141, '재판'),
   (0.019508274, '수사'),
   (0.019294487, '의혹'),
   (0.01905999, '위반'),
   (0.018494608, '부장'),
   (0.017476372, '기소'),
   (0.015745841, '판사'),
   (0.0136538055, '대해'),
   (0.012636318, '법원'),
   (0.011549418, '대한'),
   (0.010733664, '판결'),
   (0.010674046, '폭행'),
   (0.010291611, '중앙'),
   (0.0102172075, '구속'),
   (0.009297789, '검사'),
   (0.008957299, '회장')

In [ ]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'LDA_2018_하반기.html')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (-x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)
    
    
topictable = make_topictable_per_doc(model, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,13.0,0.0102,"[(2, 0.16831997), (4, 0.26818424), (7, 0.01058..."
1,1,6.0,0.0108,"[(0, 0.012670164), (1, 0.2422525), (2, 0.07691..."
2,2,19.0,0.0162,"[(2, 0.030285167), (3, 0.7322133), (4, 0.03580..."
3,3,19.0,0.0180,"[(2, 0.019563336), (3, 0.14255139), (4, 0.1880..."
4,4,13.0,0.0106,"[(0, 0.08593153), (2, 0.016399292), (3, 0.0134..."
5,5,10.0,0.0110,"[(2, 0.01821335), (4, 0.01425908), (7, 0.27588..."
6,6,16.0,0.0113,"[(2, 0.088174745), (4, 0.2809758), (9, 0.10422..."
7,7,18.0,0.0149,"[(1, 0.024875375), (2, 0.053671595), (4, 0.167..."
8,8,19.0,0.0184,"[(2, 0.1847337), (4, 0.13167165), (7, 0.390454..."
9,9,4.0,0.0127,"[(2, 0.05849798), (3, 0.22530109), (4, 0.01271..."


In [ ]:
# 토픽 갯수별 응집도 구하기

from tqdm import tqdm
from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt 

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3): 
    coherence_values = [] 
    model_list = [] 
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model) 
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence()) 
    return model_list, coherence_values 

def find_optimal_number_of_topics(dictionary, corpus, processed_data): 
    limit = 50; #토픽 마지막갯수
    start = 2; #토픽 시작갯수
    step = 6; 
    model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, 
                                                            texts=processed_data, start=start, limit=limit, step=step) 
    x = range(start, limit, step) 
    plt.plot(x, coherence_values) 
    plt.xlabel("Num Topics") 
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best') 
    plt.show() 

print('Number of unique tokens: %d' % len(dictionary)) 
print('Number of documents: %d' % len(corpus))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) 

# 최적의 토픽 수 찾기 
find_optimal_number_of_topics(dictionary, corpus, data_2018_sh)

2022-05-25 11:07:22,025 : INFO : using symmetric alpha at 0.5
2022-05-25 11:07:22,029 : INFO : using symmetric eta at 0.5
2022-05-25 11:07:22,042 : INFO : using serial LDA version on this node
2022-05-25 11:07:22,056 : INFO : running online (single-pass) LDA training, 2 topics, 1 passes over the supplied corpus of 86465 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2022-05-25 11:07:22,058 : INFO : PROGRESS: pass 0, at document #2000/86465


Number of unique tokens: 35759
Number of documents: 86465


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fro

KeyboardInterrupt: ignored

#2019

In [ ]:
first_half = data_2019['month'] <= 6
data_2019_fh = data_2019[first_half]

second_half = data_2019['month'] > 6
data_2019_sh = data_2019[second_half]

## 2019 상반기

In [ ]:
import ast
data_2019_fh_list = []
for content in data_2019_fh['noun_content']:
  content = ast.literal_eval(content)

import ast
data_2019_fh_list = []
for i in range(len(data_2019_fh)):
  tmp = data_2019_fh.iloc[i]
  c = ast.literal_eval(tmp.noun_content)
  content = [i for i in c if len(i)!=1]
  data_2019_fh_list.append(content)

dictionary = corpora.Dictionary(data_2019_fh_list)

# 빈도가 2 이상인 단어와 전체의 50%로 이상 차지하는 단어는 필터링
dictionary.filter_extremes(no_below=2, no_above=0.5)
#dictionary 속 단어가 문장에서 몇 번 출현하는지 빈도를 세서 벡터화(bag of words)이렇게 구조화된 언어를 코퍼스라고 합니다.
corpus = [dictionary.doc2bow(text) for text in data_2019_fh_list]

2022-05-25 04:20:55,883 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-25 04:20:56,343 : INFO : adding document #10000 to Dictionary(21135 unique tokens: ['거주', '결과', '관리', '교수', '명의']...)
2022-05-25 04:20:56,823 : INFO : adding document #20000 to Dictionary(28767 unique tokens: ['거주', '결과', '관리', '교수', '명의']...)
2022-05-25 04:20:57,280 : INFO : adding document #30000 to Dictionary(34023 unique tokens: ['거주', '결과', '관리', '교수', '명의']...)
2022-05-25 04:20:57,723 : INFO : adding document #40000 to Dictionary(38208 unique tokens: ['거주', '결과', '관리', '교수', '명의']...)
2022-05-25 04:20:58,188 : INFO : adding document #50000 to Dictionary(41897 unique tokens: ['거주', '결과', '관리', '교수', '명의']...)
2022-05-25 04:20:58,690 : INFO : adding document #60000 to Dictionary(44958 unique tokens: ['거주', '결과', '관리', '교수', '명의']...)
2022-05-25 04:20:59,184 : INFO : adding document #70000 to Dictionary(47711 unique tokens: ['거주', '결과', '관리', '교수', '명의']...)
2022-05-25 04:20:59,711 : INFO 

### 모델링

In [ ]:
num_topics = 20
chunksize = 2000
passes = 5
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2022-05-25 04:21:04,520 : INFO : using autotuned alpha, starting with [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
2022-05-25 04:21:04,716 : INFO : using serial LDA version on this node
2022-05-25 04:21:04,812 : INFO : running online (multi-pass) LDA training, 20 topics, 5 passes over the supplied corpus of 88587 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-05-25 04:21:04,816 : INFO : PROGRESS: pass 0, at document #2000/88587
2022-05-25 04:21:07,003 : INFO : optimized alpha [0.041110408, 0.04239501, 0.041376434, 0.043346662, 0.041378923, 0.04260244, 0.042146556, 0.04340619, 0.041100167, 0.045060348, 0.0424811, 0.04090015, 0.04217969, 0.041073825, 0.043649774, 0.0431359, 0.04214394, 0.04262967, 0.041737188, 0.04137908]
2022-05-25 04:21:07,007 : INFO : merging changes from 2000 documents into a model of 8

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-05-25 04:24:05,484 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-05-25 04:24:05,503 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-05-25 04:24:05,523 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-05-25 04:24:05,551 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-05-25 04:24:05,571 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-05-25 04:24:05,592 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-05-25 04:24:05,616 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2022-05-25 04:24:05,638 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2022-05-25 04:24:05,659 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2022-05-25 04:24:05,688 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2022-05-25 04:24:05,709 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2022-05-25 04:24:05

Average topic coherence: -4.4437.
[([(0.05237355, '미국'),
   (0.050161835, '북한'),
   (0.033728197, '대통령'),
   (0.02756407, '중국'),
   (0.025575647, '트럼프'),
   (0.020967856, '현지'),
   (0.016363846, '김정은'),
   (0.01554469, '정상회담'),
   (0.013142121, '도널드'),
   (0.0125202155, '시각'),
   (0.012413199, '정상'),
   (0.01162037, '방문'),
   (0.011507032, '국무위원'),
   (0.008897741, '시진핑'),
   (0.008896326, '일본'),
   (0.0087928055, '보도'),
   (0.0086007845, '협상'),
   (0.008054303, '시간'),
   (0.007939387, '한반도'),
   (0.0076166205, '비핵화')],
  -2.2061273395556302),
 ([(0.065661095, '혐의'),
   (0.0281938, '경찰'),
   (0.02742624, '재판'),
   (0.019277664, '선고'),
   (0.019222524, '구속'),
   (0.016173907, '기소'),
   (0.015791425, '위반'),
   (0.014745243, '검찰'),
   (0.014126879, '사건'),
   (0.014032135, '형사'),
   (0.013163588, '서울'),
   (0.012943225, '판사'),
   (0.011479574, '부장'),
   (0.011312739, '대해'),
   (0.01096276, '경찰서'),
   (0.0107833985, '불법'),
   (0.010408625, '법원'),
   (0.010308327, '남부'),
   (0.010282151, '징역

In [ ]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'LDA_2019_상반기.html')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


## 2019 하반기

In [ ]:
import ast
data_2019_sh_list = []
for content in data_2019_sh['noun_content']:
  content = ast.literal_eval(content)

import ast
data_2019_sh_list = []
for i in range(len(data_2019_sh)):
  tmp = data_2019_sh.iloc[i]
  c = ast.literal_eval(tmp.noun_content)
  content = [i for i in c if len(i)!=1]
  data_2019_sh_list.append(content)

dictionary = corpora.Dictionary(data_2019_sh_list)

# 빈도가 2 이상인 단어와 전체의 50%로 이상 차지하는 단어는 필터링
dictionary.filter_extremes(no_below=2, no_above=0.5)
#dictionary 속 단어가 문장에서 몇 번 출현하는지 빈도를 세서 벡터화(bag of words)이렇게 구조화된 언어를 코퍼스라고 합니다.
corpus = [dictionary.doc2bow(text) for text in data_2019_sh_list]

2022-05-25 04:25:11,540 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-25 04:25:11,957 : INFO : adding document #10000 to Dictionary(21144 unique tokens: ['건물', '경기도', '경찰', '고시원', '단원구']...)
2022-05-25 04:25:12,400 : INFO : adding document #20000 to Dictionary(28346 unique tokens: ['건물', '경기도', '경찰', '고시원', '단원구']...)
2022-05-25 04:25:12,839 : INFO : adding document #30000 to Dictionary(33581 unique tokens: ['건물', '경기도', '경찰', '고시원', '단원구']...)
2022-05-25 04:25:13,299 : INFO : adding document #40000 to Dictionary(37530 unique tokens: ['건물', '경기도', '경찰', '고시원', '단원구']...)
2022-05-25 04:25:13,758 : INFO : adding document #50000 to Dictionary(41122 unique tokens: ['건물', '경기도', '경찰', '고시원', '단원구']...)
2022-05-25 04:25:14,244 : INFO : adding document #60000 to Dictionary(44142 unique tokens: ['건물', '경기도', '경찰', '고시원', '단원구']...)
2022-05-25 04:25:14,735 : INFO : adding document #70000 to Dictionary(46510 unique tokens: ['건물', '경기도', '경찰', '고시원', '단원구']...)
2022-05-25

### 모델링

In [ ]:
num_topics = 20
chunksize = 2000
passes = 5
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2022-05-25 04:25:20,042 : INFO : using autotuned alpha, starting with [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
2022-05-25 04:25:20,055 : INFO : using serial LDA version on this node
2022-05-25 04:25:20,345 : INFO : running online (multi-pass) LDA training, 20 topics, 5 passes over the supplied corpus of 99466 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-05-25 04:25:20,351 : INFO : PROGRESS: pass 0, at document #2000/99466
2022-05-25 04:25:22,633 : INFO : optimized alpha [0.04143641, 0.04239312, 0.042035446, 0.041408107, 0.04185073, 0.04238721, 0.041700147, 0.042482242, 0.04205509, 0.04335369, 0.04212317, 0.04116527, 0.04440058, 0.04200545, 0.04110917, 0.042415828, 0.041670006, 0.042661563, 0.04220681, 0.04307799]
2022-05-25 04:25:22,639 : INFO : merging changes from 2000 documents into a model of 99

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-05-25 04:28:45,225 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-05-25 04:28:45,247 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-05-25 04:28:45,268 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-05-25 04:28:45,289 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-05-25 04:28:45,313 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-05-25 04:28:45,334 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-05-25 04:28:45,357 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2022-05-25 04:28:45,382 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2022-05-25 04:28:45,405 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2022-05-25 04:28:45,431 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2022-05-25 04:28:45,450 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2022-05-25 04:28:45

Average topic coherence: -5.4417.
[([(0.0914415, '상승'),
   (0.06583428, '외국인'),
   (0.065004356, '기관'),
   (0.062116202, '주가'),
   (0.061356116, '대비'),
   (0.056035228, '전일'),
   (0.052245673, '거래'),
   (0.036926866, '최근'),
   (0.03246908, '기간'),
   (0.031827725, '등락'),
   (0.028642947, '확대'),
   (0.02762794, '단위'),
   (0.024694368, '매도'),
   (0.024218421, '매수'),
   (0.020658301, '하락'),
   (0.00955742, '코스닥'),
   (0.009214606, '동향'),
   (0.00813457, '보이지'),
   (0.0076917815, '반도체'),
   (0.0074467645, '금액')],
  -2.159938543915431),
 ([(0.044612642, '한국'),
   (0.032497957, '민주당'),
   (0.03194343, '국회'),
   (0.030426677, '의원'),
   (0.029390456, '대표'),
   (0.025380619, '자유'),
   (0.014904248, '처리'),
   (0.013620098, '본회의'),
   (0.013436776, '법안'),
   (0.01278196, '선거법'),
   (0.011819906, '선거'),
   (0.011341719, '내년'),
   (0.011320877, '개정안'),
   (0.010870364, '총선'),
   (0.010791706, '미래'),
   (0.010554478, '위원'),
   (0.009138694, '원내대표'),
   (0.008874751, '위원회'),
   (0.0077666445, '협의'),
 

In [ ]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'LDA_2019_하반기.html')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


#2020

In [ ]:
first_half = data_2020['month'] <= 6
data_2020_fh = data_2020[first_half]

second_half = data_2020['month'] > 6
data_2020_sh = data_2020[second_half]

## 2020 상반기

In [ ]:
import ast
data_2020_fh_list = []
for content in data_2020_fh['noun_content']:
  content = ast.literal_eval(content)

import ast
data_2020_fh_list = []
for i in range(len(data_2020_fh)):
  tmp = data_2020_fh.iloc[i]
  c = ast.literal_eval(tmp.noun_content)
  content = [i for i in c if len(i)!=1]
  data_2020_fh_list.append(content)

dictionary = corpora.Dictionary(data_2020_fh_list)

# 빈도가 2 이상인 단어와 전체의 50%로 이상 차지하는 단어는 필터링
dictionary.filter_extremes(no_below=2, no_above=0.5)
#dictionary 속 단어가 문장에서 몇 번 출현하는지 빈도를 세서 벡터화(bag of words)이렇게 구조화된 언어를 코퍼스라고 합니다.
corpus = [dictionary.doc2bow(text) for text in data_2020_fh_list]

2022-05-25 04:30:09,221 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-25 04:30:09,656 : INFO : adding document #10000 to Dictionary(18083 unique tokens: ['고통', '공개', '관심', '구걸', '국무위원']...)
2022-05-25 04:30:10,083 : INFO : adding document #20000 to Dictionary(24222 unique tokens: ['고통', '공개', '관심', '구걸', '국무위원']...)
2022-05-25 04:30:10,529 : INFO : adding document #30000 to Dictionary(28222 unique tokens: ['고통', '공개', '관심', '구걸', '국무위원']...)
2022-05-25 04:30:10,980 : INFO : adding document #40000 to Dictionary(31282 unique tokens: ['고통', '공개', '관심', '구걸', '국무위원']...)
2022-05-25 04:30:11,407 : INFO : adding document #50000 to Dictionary(33867 unique tokens: ['고통', '공개', '관심', '구걸', '국무위원']...)
2022-05-25 04:30:11,765 : INFO : adding document #60000 to Dictionary(35279 unique tokens: ['고통', '공개', '관심', '구걸', '국무위원']...)
2022-05-25 04:30:12,166 : INFO : adding document #70000 to Dictionary(36824 unique tokens: ['고통', '공개', '관심', '구걸', '국무위원']...)
2022-05-25 04:30:

### 모델링

In [ ]:
num_topics = 20
chunksize = 2000
passes = 5
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2022-05-25 04:30:21,572 : INFO : using autotuned alpha, starting with [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
2022-05-25 04:30:21,582 : INFO : using serial LDA version on this node
2022-05-25 04:30:21,680 : INFO : running online (multi-pass) LDA training, 20 topics, 5 passes over the supplied corpus of 142751 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-05-25 04:30:21,683 : INFO : PROGRESS: pass 0, at document #2000/142751
2022-05-25 04:30:23,917 : INFO : optimized alpha [0.042840566, 0.042957053, 0.04359243, 0.04250852, 0.041266404, 0.04725816, 0.04143878, 0.041793697, 0.04120475, 0.043034103, 0.042819235, 0.04129393, 0.042749107, 0.041916817, 0.04117681, 0.04188433, 0.042714853, 0.042855587, 0.042398933, 0.04195388]
2022-05-25 04:30:23,921 : INFO : merging changes from 2000 documents into a model

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-05-25 04:34:50,047 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-05-25 04:34:50,069 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-05-25 04:34:50,091 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-05-25 04:34:50,127 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-05-25 04:34:50,146 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-05-25 04:34:50,166 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-05-25 04:34:50,185 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2022-05-25 04:34:50,207 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2022-05-25 04:34:50,228 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2022-05-25 04:34:50,250 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2022-05-25 04:34:50,271 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2022-05-25 04:34:50

Average topic coherence: -4.4983.
[([(0.07062634, '기관'),
   (0.056234792, '상승'),
   (0.054170053, '외국인'),
   (0.048910834, '주가'),
   (0.042383756, '기간'),
   (0.036953114, '거래'),
   (0.036425132, '최근'),
   (0.030636376, '연락'),
   (0.02974473, '대비'),
   (0.027186412, '매매'),
   (0.024294712, '일간'),
   (0.02381621, '원금'),
   (0.021864278, '단위'),
   (0.02167469, '전일'),
   (0.021652896, '매수'),
   (0.021591535, '매도'),
   (0.020163676, '날짜'),
   (0.019762415, '등락'),
   (0.016816204, '확대'),
   (0.01669604, '하락')],
  -2.4408486763858144),
 ([(0.03405906, '진자'),
   (0.024825972, '코로나바이러스'),
   (0.02182542, '감염'),
   (0.017943563, '발생'),
   (0.017719064, '신종'),
   (0.015921598, '판정'),
   (0.015868397, '지난'),
   (0.015808431, '확진'),
   (0.015692856, '본부'),
   (0.014433191, '환자'),
   (0.012906279, '병원'),
   (0.0120401, '마스크'),
   (0.0107369255, '신규'),
   (0.0106694335, '중앙'),
   (0.010166141, '검사'),
   (0.009885702, '관리'),
   (0.009879035, '집단'),
   (0.009518758, '의료'),
   (0.009416995, '당국'),
   (0

In [ ]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'LDA_2020_상반기.html')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


## 2020 하반기

In [ ]:
import ast
data_2020_Sh_list = []
for content in data_2020_sh['noun_content']:
  content = ast.literal_eval(content)

import ast
data_2020_Sh_list = []
for i in range(len(data_2020_sh)):
  tmp = data_2020_sh.iloc[i]
  c = ast.literal_eval(tmp.noun_content)
  content = [i for i in c if len(i)!=1]
  data_2020_Sh_list.append(content)

dictionary = corpora.Dictionary(data_2020_Sh_list)

# 빈도가 2 이상인 단어와 전체의 50%로 이상 차지하는 단어는 필터링
dictionary.filter_extremes(no_below=2, no_above=0.5)
#dictionary 속 단어가 문장에서 몇 번 출현하는지 빈도를 세서 벡터화(bag of words)이렇게 구조화된 언어를 코퍼스라고 합니다.
corpus = [dictionary.doc2bow(text) for text in data_2020_Sh_list]

2022-05-25 04:36:03,295 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-25 04:36:03,733 : INFO : adding document #10000 to Dictionary(19349 unique tokens: ['가해', '각자', '고인', '남짓', '대표']...)
2022-05-25 04:36:04,181 : INFO : adding document #20000 to Dictionary(26078 unique tokens: ['가해', '각자', '고인', '남짓', '대표']...)
2022-05-25 04:36:04,631 : INFO : adding document #30000 to Dictionary(30788 unique tokens: ['가해', '각자', '고인', '남짓', '대표']...)
2022-05-25 04:36:05,070 : INFO : adding document #40000 to Dictionary(34401 unique tokens: ['가해', '각자', '고인', '남짓', '대표']...)
2022-05-25 04:36:05,524 : INFO : adding document #50000 to Dictionary(37840 unique tokens: ['가해', '각자', '고인', '남짓', '대표']...)
2022-05-25 04:36:05,979 : INFO : adding document #60000 to Dictionary(40767 unique tokens: ['가해', '각자', '고인', '남짓', '대표']...)
2022-05-25 04:36:06,445 : INFO : adding document #70000 to Dictionary(43478 unique tokens: ['가해', '각자', '고인', '남짓', '대표']...)
2022-05-25 04:36:06,940 : INFO 

### 모델링

In [ ]:
num_topics = 20
chunksize = 2000
passes = 5
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2022-05-25 04:36:12,199 : INFO : using autotuned alpha, starting with [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
2022-05-25 04:36:12,214 : INFO : using serial LDA version on this node
2022-05-25 04:36:12,311 : INFO : running online (multi-pass) LDA training, 20 topics, 5 passes over the supplied corpus of 95627 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-05-25 04:36:12,315 : INFO : PROGRESS: pass 0, at document #2000/95627
2022-05-25 04:36:14,591 : INFO : optimized alpha [0.042377345, 0.04241165, 0.043097764, 0.04347513, 0.043309804, 0.041057482, 0.04222299, 0.04231079, 0.041782483, 0.041249424, 0.042975858, 0.041010264, 0.04189703, 0.041748602, 0.04410904, 0.04268589, 0.04118899, 0.042679645, 0.04190278, 0.042352535]
2022-05-25 04:36:14,596 : INFO : merging changes from 2000 documents into a model o

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-05-25 04:39:38,615 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-05-25 04:39:38,635 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-05-25 04:39:38,655 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-05-25 04:39:38,679 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-05-25 04:39:38,703 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-05-25 04:39:38,721 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-05-25 04:39:38,740 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2022-05-25 04:39:38,758 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2022-05-25 04:39:38,777 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2022-05-25 04:39:38,796 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2022-05-25 04:39:38,814 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2022-05-25 04:39:38

Average topic coherence: -4.8962.
[([(0.106864125, '코로나'),
   (0.03954334, '진자'),
   (0.028747413, '백신'),
   (0.019432204, '방역'),
   (0.019160807, '감염증'),
   (0.018971972, '발생'),
   (0.017888837, '신종'),
   (0.017843625, '확산'),
   (0.017644549, '감염'),
   (0.017495753, '코로나바이러스'),
   (0.016680215, '확진'),
   (0.014123612, '지난'),
   (0.013803319, '판정'),
   (0.013667296, '병원'),
   (0.012435018, '신규'),
   (0.01242859, '환자'),
   (0.010836379, '검사'),
   (0.010717983, '집단'),
   (0.0101134945, '본부'),
   (0.009783522, '국내')],
  -2.1670963749859435),
 ([(0.043481957, '법무부'),
   (0.0402305, '장관'),
   (0.0342937, '검찰'),
   (0.033569578, '검찰총장'),
   (0.03187383, '윤석열'),
   (0.024099858, '총장'),
   (0.023823094, '검사'),
   (0.02007121, '사건'),
   (0.019137593, '수사'),
   (0.018639207, '추미애'),
   (0.018171152, '대한'),
   (0.01451213, '의혹'),
   (0.014325478, '대해'),
   (0.014048476, '관련'),
   (0.012561499, '혐의'),
   (0.009336001, '법원'),
   (0.00929077, '변호사'),
   (0.009260916, '부장'),
   (0.0074299253, '청구'),


In [ ]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'LDA_2020_하반기.html')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


#2021

In [ ]:
first_half = data_2021['month'] <= 6
data_2021_fh = data_2021[first_half]

second_half = data_2021['month'] > 6
data_2021_sh = data_2021[second_half]

## 2021 상반기

In [ ]:
import ast
data_2021_fh_list = []
for content in data_2021_fh['noun_content']:
  content = ast.literal_eval(content)

import ast
data_2021_fh_list = []
for i in range(len(data_2021_fh)):
  tmp = data_2021_fh.iloc[i]
  c = ast.literal_eval(tmp.noun_content)
  content = [i for i in c if len(i)!=1]
  data_2021_fh_list.append(content)

dictionary = corpora.Dictionary(data_2021_fh_list)

# 빈도가 2 이상인 단어와 전체의 50%로 이상 차지하는 단어는 필터링
dictionary.filter_extremes(no_below=2, no_above=0.5)
#dictionary 속 단어가 문장에서 몇 번 출현하는지 빈도를 세서 벡터화(bag of words)이렇게 구조화된 언어를 코퍼스라고 합니다.
corpus = [dictionary.doc2bow(text) for text in data_2021_fh_list]

2022-05-25 04:40:38,880 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-25 04:40:39,330 : INFO : adding document #10000 to Dictionary(20405 unique tokens: ['과실', '과실치사', '과정', '관련', '구조']...)
2022-05-25 04:40:39,798 : INFO : adding document #20000 to Dictionary(27745 unique tokens: ['과실', '과실치사', '과정', '관련', '구조']...)
2022-05-25 04:40:40,284 : INFO : adding document #30000 to Dictionary(32680 unique tokens: ['과실', '과실치사', '과정', '관련', '구조']...)
2022-05-25 04:40:40,797 : INFO : adding document #40000 to Dictionary(36694 unique tokens: ['과실', '과실치사', '과정', '관련', '구조']...)
2022-05-25 04:40:41,276 : INFO : adding document #50000 to Dictionary(40131 unique tokens: ['과실', '과실치사', '과정', '관련', '구조']...)
2022-05-25 04:40:41,730 : INFO : adding document #60000 to Dictionary(42892 unique tokens: ['과실', '과실치사', '과정', '관련', '구조']...)
2022-05-25 04:40:42,178 : INFO : adding document #70000 to Dictionary(45481 unique tokens: ['과실', '과실치사', '과정', '관련', '구조']...)
2022-05-25 04:40:

### 모델링

In [ ]:
num_topics = 20
chunksize = 2000
passes = 5
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2022-05-25 04:40:45,720 : INFO : using autotuned alpha, starting with [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
2022-05-25 04:40:45,733 : INFO : using serial LDA version on this node
2022-05-25 04:40:45,824 : INFO : running online (multi-pass) LDA training, 20 topics, 5 passes over the supplied corpus of 76921 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-05-25 04:40:45,829 : INFO : PROGRESS: pass 0, at document #2000/76921
2022-05-25 04:40:48,292 : INFO : optimized alpha [0.04204628, 0.04339615, 0.042131454, 0.04253775, 0.042418167, 0.041036855, 0.041124478, 0.044988032, 0.043280058, 0.04104359, 0.042025138, 0.042167842, 0.043147326, 0.043517582, 0.041816045, 0.041247845, 0.042176053, 0.04145243, 0.042767853, 0.041867882]
2022-05-25 04:40:48,298 : INFO : merging changes from 2000 documents into a mod

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-05-25 04:43:21,820 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-05-25 04:43:21,842 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-05-25 04:43:21,863 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-05-25 04:43:21,885 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-05-25 04:43:21,907 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-05-25 04:43:21,928 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-05-25 04:43:21,949 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2022-05-25 04:43:21,971 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2022-05-25 04:43:22,001 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2022-05-25 04:43:22,022 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2022-05-25 04:43:22,046 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2022-05-25 04:43:22

Average topic coherence: -4.4035.
[([(0.08490316, '코로나'),
   (0.03351992, '백신'),
   (0.028936207, '진자'),
   (0.020889234, '방역'),
   (0.019804485, '접종'),
   (0.017975379, '신종'),
   (0.017828247, '감염증'),
   (0.017183054, '코로나바이러스'),
   (0.016760416, '감염'),
   (0.015555596, '발생'),
   (0.013405663, '신규'),
   (0.012654824, '국내'),
   (0.011974744, '집단'),
   (0.011204348, '중앙'),
   (0.010110804, '확진'),
   (0.009623574, '확산'),
   (0.009490988, '검사'),
   (0.008243217, '대책'),
   (0.0076665627, '판정'),
   (0.0072508436, '바이러스')],
  -2.218835892141726),
 ([(0.05203985, '혐의'),
   (0.028028682, '선고'),
   (0.02500709, '재판'),
   (0.019834071, '사건'),
   (0.019463347, '기소'),
   (0.019359877, '경찰'),
   (0.018604374, '형사'),
   (0.01663102, '위반'),
   (0.015658718, '징역'),
   (0.0138476435, '판사'),
   (0.013294913, '부장'),
   (0.012253923, '지법'),
   (0.012016003, '개월'),
   (0.011603175, '처벌'),
   (0.011021775, '학대'),
   (0.010699274, '정인'),
   (0.010224, '경찰청'),
   (0.009851991, '구속'),
   (0.008910316, '아동학대'),

In [ ]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'LDA_2021_상반기.html')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


## 2021 하반기

In [ ]:
import ast
data_2021_sh_list = []
for content in data_2021_sh['noun_content']:
  content = ast.literal_eval(content)

import ast
data_2021_sh_list = []
for i in range(len(data_2021_sh)):
  tmp = data_2021_sh.iloc[i]
  c = ast.literal_eval(tmp.noun_content)
  content = [i for i in c if len(i)!=1]
  data_2021_sh_list.append(content)

dictionary = corpora.Dictionary(data_2021_sh_list)

# 빈도가 2 이상인 단어와 전체의 50%로 이상 차지하는 단어는 필터링
dictionary.filter_extremes(no_below=2, no_above=0.5)
#dictionary 속 단어가 문장에서 몇 번 출현하는지 빈도를 세서 벡터화(bag of words)이렇게 구조화된 언어를 코퍼스라고 합니다.
corpus = [dictionary.doc2bow(text) for text in data_2021_sh_list]

2022-05-25 04:44:16,061 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-05-25 04:44:16,487 : INFO : adding document #10000 to Dictionary(20285 unique tokens: ['강모', '강씨', '뉴스', '대해', '돌발행동']...)
2022-05-25 04:44:16,951 : INFO : adding document #20000 to Dictionary(27362 unique tokens: ['강모', '강씨', '뉴스', '대해', '돌발행동']...)
2022-05-25 04:44:17,415 : INFO : adding document #30000 to Dictionary(32578 unique tokens: ['강모', '강씨', '뉴스', '대해', '돌발행동']...)
2022-05-25 04:44:17,902 : INFO : adding document #40000 to Dictionary(36515 unique tokens: ['강모', '강씨', '뉴스', '대해', '돌발행동']...)
2022-05-25 04:44:18,415 : INFO : adding document #50000 to Dictionary(39813 unique tokens: ['강모', '강씨', '뉴스', '대해', '돌발행동']...)
2022-05-25 04:44:18,897 : INFO : adding document #60000 to Dictionary(42747 unique tokens: ['강모', '강씨', '뉴스', '대해', '돌발행동']...)
2022-05-25 04:44:19,392 : INFO : adding document #70000 to Dictionary(45317 unique tokens: ['강모', '강씨', '뉴스', '대해', '돌발행동']...)
2022-05-25 04:44:

### 모델링

In [ ]:
num_topics = 20
chunksize = 2000
passes = 5
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2022-05-25 04:44:25,022 : INFO : using autotuned alpha, starting with [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
2022-05-25 04:44:25,034 : INFO : using serial LDA version on this node
2022-05-25 04:44:25,274 : INFO : running online (multi-pass) LDA training, 20 topics, 5 passes over the supplied corpus of 81902 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-05-25 04:44:25,284 : INFO : PROGRESS: pass 0, at document #2000/81902
2022-05-25 04:44:27,543 : INFO : optimized alpha [0.04227539, 0.042046316, 0.04204014, 0.042214207, 0.040552583, 0.04198056, 0.042541366, 0.04279241, 0.04284978, 0.041590344, 0.04096762, 0.042352557, 0.041205958, 0.042427704, 0.042933606, 0.042581875, 0.042428732, 0.041620567, 0.044798687, 0.04156559]
2022-05-25 04:44:27,551 : INFO : merging changes from 2000 documents into a model

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-05-25 04:47:11,417 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-05-25 04:47:11,438 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-05-25 04:47:11,460 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-05-25 04:47:11,481 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-05-25 04:47:11,503 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-05-25 04:47:11,526 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-05-25 04:47:11,563 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2022-05-25 04:47:11,585 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2022-05-25 04:47:11,608 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2022-05-25 04:47:11,629 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2022-05-25 04:47:11,651 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2022-05-25 04:47:11

Average topic coherence: -3.8568.
[([(0.074526355, '코로나'),
   (0.026444135, '진자'),
   (0.022991681, '백신'),
   (0.021767111, '방역'),
   (0.021140765, '접종'),
   (0.015377366, '발생'),
   (0.012410359, '신규'),
   (0.012357978, '확산'),
   (0.012007003, '신종'),
   (0.011667347, '단계'),
   (0.011429235, '감염증'),
   (0.011372088, '감염'),
   (0.010834809, '수도권'),
   (0.010802169, '코로나바이러스'),
   (0.010614627, '국내'),
   (0.01051188, '정부'),
   (0.009510883, '대책'),
   (0.009085239, '거리'),
   (0.008296613, '중앙'),
   (0.007915174, '본부')],
  -1.937557730815005),
 ([(0.06689035, '혐의'),
   (0.03108935, '선고'),
   (0.029662104, '재판'),
   (0.023200085, '기소'),
   (0.023042936, '위반'),
   (0.019461434, '부장'),
   (0.01913776, '형사'),
   (0.018137477, '징역'),
   (0.015804857, '판사'),
   (0.015712168, '사건'),
   (0.014111491, '처벌'),
   (0.012155337, '대법원'),
   (0.011845647, '범죄'),
   (0.011263398, '지법'),
   (0.010620764, '광주'),
   (0.0106159, '구속'),
   (0.010262595, '검찰'),
   (0.009828949, '법원'),
   (0.0089562535, '판결'),
  

In [ ]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'LDA_2021_하반기.html')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
data_2018.head()

,year,month,date,press,title,content,noun_content
132,2018,1,10,한겨레,"트럼프 “미국, 북한이 대화 원하면 열려있다”",도널드 트럼프 미국 대통령이 10일 문재인 대통령에게 “미국은 북한이 대화를 원할 ...,"['도널드', '트럼프', '미국', '대통령', '문재인', '대통령', '미국'..."
135,2018,1,10,경향신문,"[속보] “트럼프, 남북대화 중에는 어떤 군사적 행동도 없다”",10일 밤 문 대통령과 통화서 밝혀\n “남북대화 넘어 북 미대화 가능성에 공감”...,"['밤', '문', '대통령', '통화', '남북', '대화', '북', '미대',..."
145,2018,1,10,경향신문,"40대 환경미화원, 공사장에서 떨어진 쇠파이프에 맞아 숨져",10일 오후 3시 30분쯤 경기 고양시 덕양구의 한 아파트 신축공사장에서 쇠파이프가...,"['오후', '경기', '고양시', '덕양구', '아파트', '신축', '사장', ..."
147,2018,1,10,경향신문,[신년회견]“국회 개헌 2월까지 합의 안되면 정부안 준비”,"ㆍ문 대통령, 취임 첫 신년회견\n\n\n문재인 대통령은 10일 “국회의 합의를 기...","['문', '대통령', '취임', '첫', '신년', '회견', '문재인', '대통..."
148,2018,1,10,경향신문,[신년회견]기본권 확대 지방분권 강화 먼저 권력구조 개편은 미룰 수도,ㆍ개헌 방향\n\n\n문재인 대통령이 10일 청와대에서 열린 신년 기자회견에서 6월...,"['개헌', '방향', '문재인', '대통령', '청와대', '신년', '기자회견'..."
